# Amazom Mobiles Data Cleaning


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns

#Libraries additionally added
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import ast

#Regular Expressions
import re

# Study data files
iphone_data_path = "../Scraped Data/scraped_iphone_data.csv"
samsung_data_path = "../Scraped Data/scraped_samsung_data.csv"


# Read the Iphone & Samsung data 
iphone_df = pd.read_csv(iphone_data_path)
samsung_df = pd.read_csv(samsung_data_path)




In [2]:
#Display the Iphone DataFrame
iphone_df.head()

,title,URL,product_title,price,star_ratings,number_of_global_ratings,customer_stars_percentages,Features_1,Features_2,Features_3
0,"Apple iPhone 12, 64GB, Blue - Unlocked (Renewed)",https://www.amazon.ca/Apple-iPhone-12-64GB-Blu...,"Apple iPhone 12, 64GB, Blue - Unlocked (Renewed)",$440.00,4.1 out of 5 stars,"18,066 global ratings","{'5 star': '61%', '4 star': '15%', '3 star': '...",Apple iPhone 12,64GB,Blue - Unlocked (Renewed)
1,"Apple iPhone 12 Pro Max, 128GB, Pacific Blue -...",https://www.amazon.ca/Apple-iPhone-128GB-Pacif...,"Apple iPhone 12 Pro Max, 128GB, Pacific Blue -...",$639.00,4.1 out of 5 stars,"1,346 global ratings","{'5 star': '65%', '4 star': '12%', '3 star': '...",Apple iPhone 12 Pro Max,128GB,Pacific Blue - Unlocked (Renewed)
2,"iPhone 13, 128GB, Midnight - Unlocked (Renewed)",https://www.amazon.ca/Apple-iPhone-13-125GB-Mi...,"iPhone 13, 128GB, Midnight - Unlocked (Renewed)",$638.00,4.1 out of 5 stars,"2,120 global ratings","{'5 star': '63%', '4 star': '13%', '3 star': '...",iPhone 13,128GB,Midnight - Unlocked (Renewed)
3,"Apple iPhone 13 Pro, 128GB, Gold - Unlocked (R...",https://www.amazon.ca/Apple-iPhone-13-Pro-Rene...,"Apple iPhone 13 Pro, 128GB, Gold - Unlocked (R...",$757.52,4.2 out of 5 stars,377 global ratings,"{'5 star': '62%', '4 star': '17%', '3 star': '...",Apple iPhone 13 Pro,128GB,Gold - Unlocked (Renewed)
4,"Apple iPhone 13 Pro Max, 128GB, Sierra Blue - ...",https://www.amazon.ca/Apple-iPhone-13-Pro-Max/...,"Apple iPhone 13 Pro Max, 128GB, Sierra Blue - ...",$810.00,4.1 out of 5 stars,958 global ratings,"{'5 star': '63%', '4 star': '15%', '3 star': '...",Apple iPhone 13 Pro Max,128GB,Sierra Blue - Unlocked (Renewed)


# Clean the Iphone DataFrame

# Format, Drop, Update Columns 

In [3]:
#Check the data types
iphone_df.dtypes

title                         object
URL                           object
product_title                 object
price                         object
star_ratings                  object
number_of_global_ratings      object
customer_stars_percentages    object
Features_1                    object
Features_2                    object
Features_3                    object
dtype: object

In [4]:
# Rename the 'Features_1', 'Features_2', and 'Features_3' columns
iphone_df = iphone_df.rename(columns={
    'Features_1': 'brand_model',
    'Features_2': 'storage_capacity',
    'Features_3': 'color'
})



In [5]:

# Drop the 'title', 'product_title', and 'url' columns
iphone_df = iphone_df.drop(['title', 'product_title', 'URL'], axis=1)

# Create  'storage_capacity' column by extracting the storage capacity
iphone_df['storage_capacity'] = iphone_df['storage_capacity'].str.replace('GB', '').astype(int)

# Extract color information from the 'color' column using a regular expression
iphone_df['color'] = iphone_df['color'].str.extract(r'([a-zA-Z\s]+)').squeeze()

# Remove dollar signs and commas from the 'price' column and convert to float
iphone_df['price'] = iphone_df['price'].str.replace("$", "", regex=False).str.replace(",", "", regex=False).astype(float)

# Extract the numerical part of the 'star_ratings' column and convert to float
iphone_df['star_ratings'] = iphone_df['star_ratings'].str.split(' ', expand=True)[0].astype(float)

# Extract only the numeric values from the 'number_of_global_ratings' column
iphone_df['number_of_global_ratings'] = iphone_df['number_of_global_ratings'].str.replace(',', '')
iphone_df['number_of_global_ratings'] = iphone_df['number_of_global_ratings'].str.extract('(\d+)').astype(int)


<>:18: SyntaxWarning: invalid escape sequence '\d'
<>:18: SyntaxWarning: invalid escape sequence '\d'
C:\Users\dharm\AppData\Local\Temp\ipykernel_21672\1032329966.py:18: SyntaxWarning: invalid escape sequence '\d'
  iphone_df['number_of_global_ratings'] = iphone_df['number_of_global_ratings'].str.extract('(\d+)').astype(int)


In [6]:

# Convert string representation of dictionaries in 'customer_stars_percentages' to actual dictionaries
iphone_df['customer_stars_percentages'] = iphone_df['customer_stars_percentages'].apply(ast.literal_eval)

# Convert the 'customer_stars_percentages' column into a DataFrame with separate columns for each star rating
stars_df = pd.json_normalize(iphone_df['customer_stars_percentages'])

# Concatenate the new DataFrame with the original DataFrame
iphone_df = pd.concat([iphone_df, stars_df], axis=1)

# Drop the original 'customer_stars_percentages' columns
iphone_df = iphone_df.drop('customer_stars_percentages', axis=1)


In [7]:
# Remove percentages from '5 star' to '1 star' columns
columns_to_clean = ['5 star', '4 star', '3 star', '2 star', '1 star']

for column in columns_to_clean:
    iphone_df[column] = iphone_df[column].str.replace('%', '').astype(int)

In [8]:

# Extract "Apple" from "brand_model" and create a new "brand" column
iphone_df['brand'] = iphone_df['brand_model'].apply(lambda x: 'Apple' if 'Apple' in x else '')


# Remove "Apple" from "brand_model"
iphone_df['brand_model'] = iphone_df['brand_model'].str.replace('Apple ', '')


In [9]:

# List of manual model year values corresponding to each row
iphone_model_years = [2020, 2020,  2021, 2021, 2022, 2022, 2022]

# For iPhone dataframe
iphone_df['model_year'] = iphone_model_years


In [10]:
# Reorder the columns names as desired
desired_order = ['brand',
                 'brand_model', 
                 'storage_capacity', 
                 'color',
                 'model_year',
                 'price',
                 'number_of_global_ratings', 
                 'star_ratings', 
                 '5 star', '4 star', '3 star', '2 star', '1 star']

iphone_df = iphone_df[desired_order]




In [11]:
# Confirm the datatypes updated
iphone_df.dtypes

brand                        object
brand_model                  object
storage_capacity              int32
color                        object
model_year                    int64
price                       float64
number_of_global_ratings      int32
star_ratings                float64
5 star                        int32
4 star                        int32
3 star                        int32
2 star                        int32
1 star                        int32
dtype: object

In [12]:
# Display the resulting DataFrame
iphone_df



,brand,brand_model,storage_capacity,color,model_year,price,number_of_global_ratings,star_ratings,5 star,4 star,3 star,2 star,1 star
0,Apple,iPhone 12,64,Blue,2020,440.00,18066,4.1,61,15,6,4,14
1,Apple,iPhone 12 Pro Max,128,Pacific Blue,2020,639.00,1346,4.1,65,12,7,2,15
2,,iPhone 13,128,Midnight,2021,638.00,2120,4.1,63,13,8,6,11
3,Apple,iPhone 13 Pro,128,Gold,2021,757.52,377,4.2,62,17,7,3,10
4,Apple,iPhone 13 Pro Max,128,Sierra Blue,2022,810.00,958,4.1,63,15,7,3,12
5,Apple,iPhone 14 Pro,128,Space Black,2022,1118.28,270,4.2,65,15,6,4,9
6,Apple,iPhone 14 Pro Max,128,Deep Purple,2022,1233.29,267,4.3,73,9,3,2,12


# Samsung DataFrame

In [13]:
#Display the Samsung DataFrame
samsung_df.head()

,title,URL,product_title,price,star_ratings,number_of_global_ratings,customer_stars_percentages
0,"Samsung Galaxy S20 FE (5G) 128GB 6.5"" Display ...",https://www.amazon.ca/Samsung-Display-Factory-...,"Samsung Galaxy S20 FE (5G) 128GB 6.5"" Display ...",$240.00,3.9 out of 5 stars,908 global ratings,"{'5 star': '58%', '4 star': '13%', '3 star': '..."
1,"Samsung Galaxy S20 Ultra, 128GB, Cosmic Gray -...",https://www.amazon.ca/Samsung-Galaxy-S20-Unloc...,"Samsung Galaxy S20 Ultra, 128GB, Cosmic Gray -...",$437.99,4.0 out of 5 stars,"1,732 global ratings","{'5 star': '60%', '4 star': '13%', '3 star': '..."
2,"Samsung Galaxy S21 5G 128GB 6.2"" Display, Snap...",https://www.amazon.ca/Samsung-Galaxy-S21-5G-Sn...,"Samsung Galaxy S21 5G 128GB 6.2"" Display, Snap...",$298.88,3.9 out of 5 stars,403 global ratings,"{'5 star': '55%', '4 star': '17%', '3 star': '..."
3,Samsung Galaxy S21 Ultra (5G) 128GB Unlocked -...,https://www.amazon.ca/Samsung-Galaxy-S21-Ultra...,Samsung Galaxy S21 Ultra (5G) 128GB Unlocked -...,$469.99,3.8 out of 5 stars,246 global ratings,"{'5 star': '56%', '4 star': '15%', '3 star': '..."
4,Samsung Galaxy S22 ULTRA (5G) 128GB Unlocked -...,https://www.amazon.ca/Samsung-Galaxy-Ultra-128...,Samsung Galaxy S22 ULTRA (5G) 128GB Unlocked -...,$618.00,4.0 out of 5 stars,781 global ratings,"{'5 star': '60%', '4 star': '14%', '3 star': '..."


# Clean the Samsung  DataFrame

# Format, Drop, Update Columns 

In [14]:
#Check the data types
samsung_df.dtypes

title                         object
URL                           object
product_title                 object
price                         object
star_ratings                  object
number_of_global_ratings      object
customer_stars_percentages    object
dtype: object

In [15]:

def extract_phone_model(url):
    match = re.search(r'Samsung[\w\s\-\(\)]+(?=\d+GB)', url)
    return match.group(0).replace('-', ' ').strip() if match else 'Unknown'

def extract_size(url):
    match = re.search(r'\d+GB', url)
    return match.group(0) if match else 'Unknown'


def extract_color(url):
    colors = ['Black', 'Navy', 'White', 'Red', 'Blue', 'Green', 'Yellow', 'Purple', 'Pink', 'Grey', 'Silver', 'Gold', 'Bronze']
    for color in colors:
        if color.lower() in url.lower():
            return color
    return 'Unknown'

In [16]:
samsung_df['titlenew']=samsung_df['title'].apply(extract_phone_model).str.split().str[0:4].apply(lambda x: ' '.join(x))

In [17]:
samsung_df['size']=samsung_df['title'].apply(extract_size).str[0:3].astype('int64')

In [18]:
samsung_df['color']=samsung_df['title'].apply(extract_color)


In [19]:

# Drop the 'title', 'product_title', and 'url' columns
samsung_df = samsung_df.drop(['title', 'product_title', 'URL'], axis=1)


# Remove dollar signs and commas from the 'price' column and convert to float
samsung_df['price'] = samsung_df['price'].str.replace("$", "", regex=False).str.replace(",", "", regex=False).astype(float)

# Extract the numerical part of the 'star_ratings' column and convert to float
samsung_df['star_ratings'] = samsung_df['star_ratings'].str.split(' ', expand=True)[0].astype(float)

# Extract only the numeric values from the 'number_of_global_ratings' column
samsung_df['number_of_global_ratings'] = samsung_df['number_of_global_ratings'].str.replace(',', '')
samsung_df['number_of_global_ratings'] = samsung_df['number_of_global_ratings'].str.extract('(\d+)').astype(int)


<>:13: SyntaxWarning: invalid escape sequence '\d'
<>:13: SyntaxWarning: invalid escape sequence '\d'
C:\Users\dharm\AppData\Local\Temp\ipykernel_21672\2822677854.py:13: SyntaxWarning: invalid escape sequence '\d'
  samsung_df['number_of_global_ratings'] = samsung_df['number_of_global_ratings'].str.extract('(\d+)').astype(int)


In [20]:

# Convert string representation of dictionaries in 'customer_stars_percentages' to actual dictionaries
samsung_df['customer_stars_percentages'] = samsung_df['customer_stars_percentages'].apply(ast.literal_eval)

# Convert the 'customer_stars_percentages' column into a DataFrame with separate columns for each star rating
stars_df = pd.json_normalize(samsung_df['customer_stars_percentages'])

# Concatenate the new DataFrame with the original DataFrame
samsung_df = pd.concat([samsung_df, stars_df], axis=1)

# Drop the original 'customer_stars_percentages' column
samsung_df = samsung_df.drop('customer_stars_percentages', axis=1)



In [21]:
# Remove percentages from '5 star' to '1 star' columns
columns_to_clean = ['5 star', '4 star', '3 star', '2 star', '1 star']

for column in columns_to_clean:
    samsung_df[column] = samsung_df[column].str.replace('%', '').astype(int)

In [22]:

# Rename the 'titlenew' and 'size' columns
samsung_df= samsung_df.rename(columns={
    'titlenew': 'brand_model',
    'size': 'storage_capacity'
})



In [23]:

# Extract "Samsung" from "brand_model" and create a new "brand" column
samsung_df['brand'] = samsung_df['brand_model'].apply(lambda x: 'Samsung' if 'Samsung' in x else '')


# Remove "Apple" from "brand_model"
samsung_df['brand_model'] = samsung_df['brand_model'].str.replace('Samsung ', '')


In [24]:

# List of manual model year values corresponding to each row
samsung_model_years = [2020, 2021, 2022, 2022, 2023, 2023]

# For Samsung dataframe
samsung_df['model_year'] = samsung_model_years

In [25]:

# Reorder the columns names as desired
desired_order = ['brand',
                 'brand_model', 
                 'storage_capacity', 
                 'color',
                 'model_year',
                 'price', 
                 'number_of_global_ratings', 
                 'star_ratings', 
                 '5 star', '4 star', '3 star', '2 star', '1 star']

samsung_df = samsung_df[desired_order]



In [26]:
# Confirm the updated data types 
samsung_df.dtypes

brand                        object
brand_model                  object
storage_capacity              int64
color                        object
model_year                    int64
price                       float64
number_of_global_ratings      int32
star_ratings                float64
5 star                        int32
4 star                        int32
3 star                        int32
2 star                        int32
1 star                        int32
dtype: object

In [27]:
# Display the resulting DataFrame
samsung_df

,brand,brand_model,storage_capacity,color,model_year,price,number_of_global_ratings,star_ratings,5 star,4 star,3 star,2 star,1 star
0,Samsung,Galaxy S20 FE,128,Navy,2020,240.00,908,3.9,58,13,6,5,17
1,,Unknown,128,Unknown,2021,437.99,1732,4.0,60,13,7,5,15
2,Samsung,Galaxy S21 5G,128,Grey,2022,298.88,403,3.9,55,17,7,7,14
3,Samsung,Galaxy S21 Ultra,128,Black,2022,469.99,246,3.8,56,15,6,4,19
4,Samsung,Galaxy S22 ULTRA,128,Black,2023,618.00,781,4.0,60,14,6,5,16
5,,Unknown,256,Black,2023,998.98,162,4.2,72,5,4,6,14


# Save iphone_df and samsung_df to JSON and CSV 

In [28]:
samsung_df.to_json('../Cleaned Data/samsung_cleaned_data.json', orient='records', lines=True)
samsung_df.to_csv('../Cleaned Data/samsung_cleaned_data.csv', index=False)

In [29]:
iphone_df.to_json('../Cleaned Data/iphone_cleaned_data.json', orient='records', lines=True)
iphone_df.to_csv('../Cleaned Data/iphone_cleaned_data.csv', index=False)

In [30]:
iphone_df


,brand,brand_model,storage_capacity,color,model_year,price,number_of_global_ratings,star_ratings,5 star,4 star,3 star,2 star,1 star
0,Apple,iPhone 12,64,Blue,2020,440.00,18066,4.1,61,15,6,4,14
1,Apple,iPhone 12 Pro Max,128,Pacific Blue,2020,639.00,1346,4.1,65,12,7,2,15
2,,iPhone 13,128,Midnight,2021,638.00,2120,4.1,63,13,8,6,11
3,Apple,iPhone 13 Pro,128,Gold,2021,757.52,377,4.2,62,17,7,3,10
4,Apple,iPhone 13 Pro Max,128,Sierra Blue,2022,810.00,958,4.1,63,15,7,3,12
5,Apple,iPhone 14 Pro,128,Space Black,2022,1118.28,270,4.2,65,15,6,4,9
6,Apple,iPhone 14 Pro Max,128,Deep Purple,2022,1233.29,267,4.3,73,9,3,2,12


In [31]:
samsung_df

,brand,brand_model,storage_capacity,color,model_year,price,number_of_global_ratings,star_ratings,5 star,4 star,3 star,2 star,1 star
0,Samsung,Galaxy S20 FE,128,Navy,2020,240.00,908,3.9,58,13,6,5,17
1,,Unknown,128,Unknown,2021,437.99,1732,4.0,60,13,7,5,15
2,Samsung,Galaxy S21 5G,128,Grey,2022,298.88,403,3.9,55,17,7,7,14
3,Samsung,Galaxy S21 Ultra,128,Black,2022,469.99,246,3.8,56,15,6,4,19
4,Samsung,Galaxy S22 ULTRA,128,Black,2023,618.00,781,4.0,60,14,6,5,16
5,,Unknown,256,Black,2023,998.98,162,4.2,72,5,4,6,14


In [32]:
# Concatenate the dataframes along the rows
iphone_samsung_df = pd.concat([iphone_df, samsung_df], ignore_index=True)

iphone_samsung_df

,brand,brand_model,storage_capacity,color,model_year,price,number_of_global_ratings,star_ratings,5 star,4 star,3 star,2 star,1 star
0,Apple,iPhone 12,64,Blue,2020,440.00,18066,4.1,61,15,6,4,14
1,Apple,iPhone 12 Pro Max,128,Pacific Blue,2020,639.00,1346,4.1,65,12,7,2,15
2,,iPhone 13,128,Midnight,2021,638.00,2120,4.1,63,13,8,6,11
3,Apple,iPhone 13 Pro,128,Gold,2021,757.52,377,4.2,62,17,7,3,10
4,Apple,iPhone 13 Pro Max,128,Sierra Blue,2022,810.00,958,4.1,63,15,7,3,12
5,Apple,iPhone 14 Pro,128,Space Black,2022,1118.28,270,4.2,65,15,6,4,9
6,Apple,iPhone 14 Pro Max,128,Deep Purple,2022,1233.29,267,4.3,73,9,3,2,12
7,Samsung,Galaxy S20 FE,128,Navy,2020,240.00,908,3.9,58,13,6,5,17
8,,Unknown,128,Unknown,2021,437.99,1732,4.0,60,13,7,5,15
9,Samsung,Galaxy S21 5G,128,Grey,2022,298.88,403,3.9,55,17,7,7,14


In [33]:

iphone_samsung_df.to_json('../Cleaned Data/iphone_samsung_merged_data.json', orient='records', lines=True)
iphone_samsung_df.to_csv('../Cleaned Data/iphone_samsung_merged_data.csv', index=False)